In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import time

In [3]:
# Konfigurasi
#video_path = 'D:\\TA_Lalu_Lintas\\TA-Lalu-Lintas\\dataset-mentah\\vidios\\TC_00005.mov'  # Path ke video
video_path = 'C:\\TA-Lalu-Lintas\\dataset-mentah\\vidios\\TC_00005.mov'
image_folder = 'D:\\TA_Lalu_Lintas\\TA-Lalu-Lintas\\dataset-mentah\\images' 

# Direktori untuk dataset YOLO
output_yolo_images = 'dataset_yolo/images/'
output_yolo_labels = 'dataset_yolo/labels/'
os.makedirs(output_yolo_images, exist_ok=True)
os.makedirs(output_yolo_labels, exist_ok=True)

# Direktori untuk dataset CNN (gambar kepala)
output_cnn_worn = 'dataset_cnn/helmet_worn/'
output_cnn_not_worn = 'dataset_cnn/helmet_not_worn/'
os.makedirs(output_cnn_worn, exist_ok=True)
os.makedirs(output_cnn_not_worn, exist_ok=True)

In [4]:
# Variabel Global
drawing = False
ix, iy = -1, -1
boxes = []  # Menyimpan semua bounding box per frame
current_class = 0  # 0 = Motor, 1 = Helmet
frame_count = 0
paused = False  # Status video (berjalan atau berhenti)
# FRAMES PER SECOND FOR VIDEO
fps = 30

In [5]:
# Fungsi menggambar bounding box
import cv2
def draw_bbox(event, x, y, flags, param):
    global ix, iy, drawing, img, boxes, current_class

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            img_copy = img.copy()
            color = (0, 255, 0) if current_class == 0 else (255, 0, 0)
            cv2.rectangle(img_copy, (ix, iy), (x, y), color, 2)
            cv2.imshow('Frame', img_copy)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        color = (0, 255, 0) if current_class == 0 else (255, 0, 0)
        cv2.rectangle(img, (ix, iy), (x, y), color, 2)
        x_min, y_min = min(ix, x), min(iy, y)
        x_max, y_max = max(ix, x), max(iy, y)
        boxes.append((current_class, x_min, y_min, x_max, y_max))
        cv2.imshow('Frame', img)

In [6]:
# Fungsi untuk mendapatkan nomor urut terakhir dari file dalam sebuah folder
def get_last_file_number(folder_path, prefix, extension):
    """
    Mengambil nomor urut terakhir dari file dengan prefix tertentu dalam folder.
    Contoh file: 'frame_0001.txt' untuk label YOLO atau 'head_0001.jpg' untuk gambar kepala
    
    Args:
    - folder_path: Lokasi folder tempat file disimpan
    - prefix: Prefix dari file (misalnya 'frame' atau 'head')
    - extension: Ekstensi file (misalnya '.txt' untuk label atau '.jpg' untuk gambar)
    
    Returns:
    - nomor urut terakhir yang ditemukan, jika tidak ada file maka 0
    """
    # Ambil semua file dalam folder
    files = os.listdir(folder_path)
    
    # Filter file yang memiliki prefix dan ekstensi tertentu
    pattern = re.compile(f'{prefix}_(\d+){extension}')  # Filter dengan ekstensi sesuai (txt untuk label, jpg untuk gambar)
    numbers = []
    
    for file in files:
        match = pattern.match(file)
        if match:
            numbers.append(int(match.group(1)))  # Ambil nomor urut dari nama file
    
    # Jika tidak ada file yang sesuai, kembalikan 0
    if not numbers:
        return 0
    
    # Kembalikan nomor urut terbesar
    return max(numbers)


In [ ]:
# Load video
cap = cv2.VideoCapture(video_path)
cv2.namedWindow('Frame')
cv2.setMouseCallback('Frame', draw_bbox)

frame_count = 0
paused = False
boxes = []  # Temporary list for storing current bounding boxes
saved_bboxes = set()  # Set to store already saved bounding boxes

while cap.isOpened():
    if not paused:
        time.sleep(1/fps)
        ret, frame = cap.read()
        if not ret:
            print("Video selesai!")
            break
        img = frame.copy()
        cv2.imshow('Frame', img)

    key = cv2.waitKey(1)

    if key == 32:  # Space untuk pause/play
        paused = not paused

    elif key == ord('m'):  # Motor
        current_class = 0

    elif key == ord('h'):  # Helmet
        current_class = 1

    elif key == ord('r'):  # Reset label
        img = frame.copy()
        boxes.clear()

    elif key == ord('n'):  # Next frame tanpa menyimpan
        paused = False
        boxes.clear()

    elif key == ord('q'):  # Keluar program
        cap.release()
        cv2.destroyAllWindows()
        exit()

    elif key == 13:  # Simpan data jika ada label
        if len(boxes) == 0:
            print("Tidak ada label, frame tidak disimpan!")
        else:
            # Dapatkan nomor terakhir dari folder
            last_label_number = get_last_file_number(output_yolo_labels, "frame", ".txt")  # Untuk label YOLO
            last_image_number = get_last_file_number(output_yolo_images, "frame", ".jpg")  # Untuk gambar YOLO
            last_worn = get_last_file_number(output_cnn_worn, "head", ".jpg")  # Untuk helm yang dikenakan
            last_not_worn = get_last_file_number(output_cnn_not_worn, "head", ".jpg")  # Untuk helm yang tidak dikenakan

            # Simpan label dalam format YOLO
            label_file = os.path.join(output_yolo_labels, f'frame_{last_label_number + 1:04d}.txt')  # Perbaikan di sini
            with open(label_file, 'w') as f:
                motor_bboxes = []  # Menyimpan semua bounding box motor
                helmet_bboxes = []  # Menyimpan semua bounding box helm

                # Pisahkan motor dan helm
                for class_id, x_min, y_min, x_max, y_max in boxes:
                    # Memeriksa apakah bounding box sudah pernah disimpan
                    bbox_key = (class_id, x_min, y_min, x_max, y_max)
                    if bbox_key in saved_bboxes:
                        continue  # Skip jika sudah disimpan

                    # Konversi koordinat ke format YOLO
                    x_center = (x_min + x_max) / 2 / frame.shape[1]
                    y_center = (y_min + y_max) / 2 / frame.shape[0]
                    width = (x_max - x_min) / frame.shape[1]
                    height = (y_max - y_min) / frame.shape[0]
                    f.write(f'{class_id} {x_center} {y_center} {width} {height}\n')

                    if class_id == 0:  # Motor
                        motor_bboxes.append((x_min, y_min, x_max, y_max))
                    elif class_id == 1:  # Helmet
                        helmet_bboxes.append((x_min, y_min, x_max, y_max))

                    # Tandai bounding box sebagai sudah disimpan
                    saved_bboxes.add(bbox_key)

            # Simpan gambar YOLO
            image_file = os.path.join(output_yolo_images, f'frame_{last_image_number + 1:04d}.jpg')
            cv2.imwrite(image_file, frame)

            # Ekstrak kepala untuk dataset CNN
            for motor_bbox in motor_bboxes:
                x_min, y_min, x_max, y_max = motor_bbox
                head_crop = frame[y_min:y_min + int(0.3 * (y_max - y_min)), x_min:x_max]

                # Memeriksa apakah helm ada di dalam bounding box motor
                helmet_found = False
                for helmet_bbox in helmet_bboxes:
                    hx_min, hy_min, hx_max, hy_max = helmet_bbox
                    if hx_min >= x_min and hy_min >= y_min and hx_max <= x_max and hy_max <= y_max:
                        helmet_found = True
                        break

                # Simpan gambar CNN berdasarkan status helm
                if helmet_found:
                    last_worn += 1
                    filename = f'head_{last_worn:04d}.jpg'
                    cv2.imwrite(os.path.join(output_cnn_worn, filename), head_crop)
                else:
                    last_not_worn += 1
                    filename = f'head_{last_not_worn:04d}.jpg'
                    cv2.imwrite(os.path.join(output_cnn_not_worn, filename), head_crop)

            frame_count += 1
            paused = False

cap.release()
cv2.destroyAllWindows()


: 

In [4]:
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from PIL import Image
import os
import numpy as np
import datetime

# ==== Preprocessing Functions ====
def remove_noise(image):
    return cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 21)

def enhance_contrast(image):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    lab = cv2.merge((cl, a, b))
    return cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

def sharpen_image(image):
    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]])
    return cv2.filter2D(image, -1, kernel)

# ==== Configuration ====
YOLO_MODEL_PATH = "bestYolo11.pt"
CNN_MODEL_PATH = "best_model_efficientnet.pth"
VIDEO_PATH = "C:\\TA-Lalu-Lintas\\dataset-mentah\\vidios\\IMG_3342.MOV"
SAVE_DIR = "pelanggar"
MIN_BOX_AREA = 90000
CNN_CONFIDENCE_THRESHOLD = 0.8
HEAD_CROP_RATIO = 0.4  # Bagian atas bounding box untuk crop kepala

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==== Load YOLO Model ====
yolo_model = YOLO(YOLO_MODEL_PATH).to(device)

# ==== Load EfficientNetB0 Model ====
def load_cnn_model():
    model = models.efficientnet_b0(pretrained=False)
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)
    model.load_state_dict(torch.load(CNN_MODEL_PATH, map_location=device))
    return model.to(device).eval()

cnn_model = load_cnn_model()

cnn_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

os.makedirs(SAVE_DIR, exist_ok=True)

# ==== Tracker ====
tracker = DeepSort(max_age=15, n_init=3, max_cosine_distance=0.4, nn_budget=100)

# ==== Pelacakan Pelanggar ====
processed_ids = set()
HELMET_LABELS = ["Helm", "Head"]  # 0 = pakai, 1 = tidak pakai

# ==== Video Processing ====
cap = cv2.VideoCapture(VIDEO_PATH)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = yolo_model(frame, verbose=False)[0]
    detections = []

    # Ambil deteksi pengendara motor (kelas 0)
    for box in results.boxes:
        cls = int(box.cls[0].item())
        if cls == 0:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = box.conf[0].item()
            w, h = x2 - x1, y2 - y1
            detections.append(([x1, y1, w, h], conf, "Pengendara"))

    # Update tracker
    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed() or track.time_since_update > 0:
            continue

        track_id = track.track_id
        x1, y1, x2, y2 = map(int, track.to_ltrb())
        conf = track.det_conf if hasattr(track, 'det_conf') else 0.0

        # Tampilkan bounding box untuk semua deteksi
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 2)
        label_text = f"ID {track_id} | YOLO:{conf:.2f}"
        (text_w, text_h), _ = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
        cv2.rectangle(frame, (x1, y1 - text_h - 10), (x1 + text_w + 6, y1), (0, 255, 255), -1)
        cv2.putText(frame, label_text, (x1 + 3, y1 - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)

        box_area = (x2 - x1) * (y2 - y1)
        if box_area < MIN_BOX_AREA or track_id in processed_ids:
            continue

        # Crop full body
        cropped_body = frame[y1:y2, x1:x2]
        if cropped_body.size == 0:
            continue

        # Crop kepala dari bagian atas bounding box
        head_h = int((y2 - y1) * HEAD_CROP_RATIO)
        head_crop = frame[y1:y1 + head_h, x1:x2]
        if head_crop.size == 0:
            continue

        try:
            img = cv2.resize(head_crop, (224, 224))
            img = remove_noise(img)
            img = sharpen_image(img)
            img = enhance_contrast(img)
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img_pil = Image.fromarray(img_rgb)
            img_tensor = cnn_transform(img_pil).unsqueeze(0).to(device)
        except Exception as e:
            print(f"Preprocessing error: {e}")
            continue

        with torch.no_grad():
            outputs = cnn_model(img_tensor)
            probs = torch.nn.functional.softmax(outputs, dim=1)
            _, preds = torch.max(outputs, 1)

        label_idx = preds.item()
        cnn_conf = probs[0][label_idx].item()
        processed_ids.add(track_id)

        # Simpan pelanggaran jika tidak pakai helm
        if label_idx == 1 and cnn_conf > CNN_CONFIDENCE_THRESHOLD:
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S%f")
            save_path = os.path.join(SAVE_DIR, f"violation_ID{track_id}_{timestamp}.jpg")
            cv2.imwrite(save_path, cropped_body)
            print(f"[!] Pelanggaran disimpan: ID {track_id} | CNN Conf: {cnn_conf:.2f}")

    # Tampilkan frame
    cv2.imshow('E-Tilang', frame)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

C:\Users\SER3\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\SER3\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
